In [12]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/Colab Notebooks/초미세먼지 오염도 단기 예측 모델/' # 구글 클라우드 데이터 위치

Mounted at /content/drive


In [3]:
test_aws = pd.read_csv(path+"preprocessed_test.csv")

In [4]:
test_aws

,연도,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),지역번호,월,일,시,fix_기온(°C),fix_습도(%),fix_강수량(mm),fix_풍속(m/s),fix_풍향(deg),fix_월,fix_일,fix_시
0,4,옹도,0.325434,0.071944,0.248082,0.000000,0.680,3,1,1,0,1,2,0,7.0,1.0,3,1,2
1,4,옹도,0.322275,0.070833,0.253197,0.000000,0.667,3,1,1,1,1,2,0,7.0,1.0,3,1,2
2,4,옹도,0.320695,0.098056,0.237852,0.000000,0.668,3,1,1,2,1,2,0,7.0,1.0,3,1,1
3,4,옹도,0.320695,0.055833,0.276215,0.000000,0.658,3,1,1,3,1,2,0,7.0,1.0,3,1,1
4,4,옹도,0.323855,0.040556,0.230179,0.000000,0.671,3,1,1,4,1,2,0,7.0,1.0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231835,4,정안,0.429700,0.819167,0.099744,0.000617,0.363,3,11,18,19,1,0,1,5.0,7.0,3,1,2
231836,4,정안,0.413902,0.788333,0.092072,0.000653,0.525,3,11,18,20,1,2,1,5.0,7.0,3,1,2
231837,4,정안,0.398104,0.789167,0.102302,0.000768,0.495,3,11,18,21,1,0,1,6.0,7.0,3,1,2
231838,4,정안,0.390205,0.810556,0.094629,0.000000,0.600,3,11,18,22,1,2,0,5.0,7.0,3,1,2


In [19]:
test_aws.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231840 entries, 0 to 231839
Data columns (total 19 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   연도           231840 non-null  int64  
 1   지점           231840 non-null  object 
 2   기온(°C)       231840 non-null  float64
 3   풍향(deg)      231840 non-null  float64
 4   풍속(m/s)      231840 non-null  float64
 5   강수량(mm)      231840 non-null  float64
 6   습도(%)        231840 non-null  float64
 7   지역번호         231840 non-null  int64  
 8   월            231840 non-null  int64  
 9   일            231840 non-null  int64  
 10  시            231840 non-null  int64  
 11  fix_기온(°C)   231840 non-null  int64  
 12  fix_습도(%)    231840 non-null  int64  
 13  fix_강수량(mm)  231840 non-null  int64  
 14  fix_풍속(m/s)  231840 non-null  float64
 15  fix_풍향(deg)  231840 non-null  float64
 16  fix_월        231840 non-null  int64  
 17  fix_일        231840 non-null  int64  
 18  fix_시        231840 non-

In [28]:
test_pm_path = path+"TEST_INPUT/"
test_pm_list = os.listdir(test_pm_path)

In [14]:
print(len(test_pm_list))

17


In [15]:
test_pm_list

['공주.csv',
 '노은동.csv',
 '논산.csv',
 '대천2동.csv',
 '독곶리.csv',
 '동문동.csv',
 '모종동.csv',
 '문창동.csv',
 '성성동.csv',
 '신방동.csv',
 '신흥동.csv',
 '아름동.csv',
 '예산군.csv',
 '읍내동.csv',
 '이원면.csv',
 '정림동.csv',
 '홍성읍.csv']

In [42]:
merged_pm_test = pd.read_csv(test_pm_path + test_pm_list[0])
for i in range(1, len(test_pm_list)):
  tmp = pd.read_csv(test_pm_path + test_pm_list[i])
  merged_pm_test = pd.concat([merged_pm_test, tmp])

In [43]:
merged_pm_test.reset_index(inplace = True, drop = True)
merged_pm_test

,연도,일시,측정소,PM2.5
0,4,01-01 00:00,공주,0.060
1,4,01-01 01:00,공주,0.064
2,4,01-01 02:00,공주,0.072
3,4,01-01 03:00,공주,0.064
4,4,01-01 04:00,공주,0.056
...,...,...,...,...
131371,4,11-18 19:00,홍성읍,0.056
131372,4,11-18 20:00,홍성읍,0.036
131373,4,11-18 21:00,홍성읍,0.020
131374,4,11-18 22:00,홍성읍,0.024


In [44]:
# 일시 컬럼 이용해 월, 일, 시, 분 단위로 컬럼 split
def date_split(df):
    df['날짜'] = df.일시.str.split().str[0]
    df['시간'] = df.일시.str.split().str[1]
    df['월'] = df.날짜.str.split('-').str[0]
    df['일'] = df.날짜.str.split('-').str[1]
    df['시'] = df.시간.str.split(':').str[0]
    
    # 1시간마다 측정했으므로 분 단위 필요 X
    # df['분'] = df.시간.str.split(':').str[1]
    
    # 분할했으니 기존 컬럼 제거
    df.drop(['날짜', '시간', '일시'],axis = 1, inplace=True)
    
    # Split 진행한 컬럼들 형변환
    df['월'] = df['월'].astype('int64')
    df['일'] = df['일'].astype('int64')
    df['시'] = df['시'].astype('int64')
    return df

pm_test = date_split(merged_pm_test)
pm_test

,연도,측정소,PM2.5,월,일,시
0,4,공주,0.060,1,1,0
1,4,공주,0.064,1,1,1
2,4,공주,0.072,1,1,2
3,4,공주,0.064,1,1,3
4,4,공주,0.056,1,1,4
...,...,...,...,...,...,...
131371,4,홍성읍,0.056,11,18,19
131372,4,홍성읍,0.036,11,18,20
131373,4,홍성읍,0.020,11,18,21
131374,4,홍성읍,0.024,11,18,22


In [20]:
pm_dict = {'아름동': '세종고운',
             '신흥동': '세종연서',
             '노은동': '계룡',
             '문창동': '오월드',
             '읍내동': '장동',
             '정림동': '오월드',
             '공주': '공주',
             '논산': '논산',
             '대천2동': '대천항',
             '독곶리': '대산',
             '동문동': '태안',
             '모종동': '아산',
             '신방동': '성거',
             '예산군': '예산',
             '이원면': '태안',
             '홍성읍': '홍북',
             '성성동': '성거'}

In [47]:
pm_test

,연도,측정소,PM2.5,월,일,시,지점
0,4,공주,0.060,1,1,0,
1,4,공주,0.064,1,1,1,
2,4,공주,0.072,1,1,2,
3,4,공주,0.064,1,1,3,
4,4,공주,0.056,1,1,4,
...,...,...,...,...,...,...,...
131371,4,홍성읍,0.056,11,18,19,
131372,4,홍성읍,0.036,11,18,20,
131373,4,홍성읍,0.020,11,18,21,
131374,4,홍성읍,0.024,11,18,22,


In [64]:
# pm_list = list(pm_dict.keys())
# aws_list = list(pm_dict.values())

pm_test['지점'] = ''
for i in range(len(pm_test)):
  pm_test.loc[i, "지점"] = pm_dict[pm_test.loc[i, "측정소"]]
pm_test

,연도,측정소,PM2.5,월,일,시,지점
0,4,공주,0.060,1,1,0,공주
1,4,공주,0.064,1,1,1,공주
2,4,공주,0.072,1,1,2,공주
3,4,공주,0.064,1,1,3,공주
4,4,공주,0.056,1,1,4,공주
...,...,...,...,...,...,...,...
131371,4,홍성읍,0.056,11,18,19,홍북
131372,4,홍성읍,0.036,11,18,20,홍북
131373,4,홍성읍,0.020,11,18,21,홍북
131374,4,홍성읍,0.024,11,18,22,홍북


In [65]:
test = pm_test.merge(test_aws, how = 'left', on = ['연도', '월', '일', '시', '지점'])

In [75]:
train = pd.read_csv(path + "makedummy_train.csv")
train

,연도,측정소,PM2.5,지점,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%),지역번호,월,일,시,fix_월,fix_일,fix_시,fix_기온(°C),fix_습도(%),fix_강수량(mm)
0,0,아름동,0.044,세종고운,0.170616,0.053333,0.017903,0.0,0.764,1,1,1,0,3,1,2,0,2,0
1,0,아름동,0.048,세종고운,0.167457,0.099444,0.020460,0.0,0.759,1,1,1,1,3,1,2,0,2,0
2,0,아름동,0.060,세종고운,0.162717,0.134167,0.012788,0.0,0.760,1,1,1,2,3,1,1,0,2,0
3,0,아름동,0.064,세종고운,0.157978,0.000000,0.005115,0.0,0.747,1,1,1,3,3,1,1,0,2,0
4,0,아름동,0.064,세종고운,0.148499,0.000000,0.007673,0.0,0.791,1,1,1,4,3,1,1,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580541,3,홍성읍,0.060,홍북,0.273302,0.832222,0.086957,0.0,0.671,3,12,31,19,3,1,2,0,2,0
580542,3,홍성읍,0.052,홍북,0.271722,0.831667,0.043478,0.0,0.692,3,12,31,20,3,1,2,0,2,0
580543,3,홍성읍,0.044,홍북,0.268562,0.832500,0.066496,0.0,0.706,3,12,31,21,3,1,2,0,2,0
580544,3,홍성읍,0.052,홍북,0.262243,0.866944,0.043478,0.0,0.725,3,12,31,22,3,1,2,0,2,0


In [79]:
test

,연도,측정소,PM2.5,월,일,시,지점,기온(°C),풍향(deg),풍속(m/s),...,습도(%),지역번호,fix_기온(°C),fix_습도(%),fix_강수량(mm),fix_풍속(m/s),fix_풍향(deg),fix_월,fix_일,fix_시
0,4,공주,0.060,1,1,0,공주,0.244866,0.123333,0.038363,...,0.647,3,0,2,1,4.0,1.0,3,1,2
1,4,공주,0.064,1,1,1,공주,0.232227,0.167778,0.033248,...,0.648,3,0,2,0,3.0,2.0,3,1,2
2,4,공주,0.072,1,1,2,공주,0.206951,0.238621,0.002558,...,0.734,3,0,2,0,1.0,2.0,3,1,1
3,4,공주,0.064,1,1,3,공주,0.199052,0.240016,0.002558,...,0.753,3,0,2,0,1.0,2.0,3,1,1
4,4,공주,0.056,1,1,4,공주,0.189573,0.342192,0.002558,...,0.795,3,0,2,1,1.0,3.0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139099,4,홍성읍,0.056,11,18,19,홍북,0.404423,0.833611,0.089514,...,0.554,3,1,2,1,5.0,7.0,3,1,2
139100,4,홍성읍,0.036,11,18,20,홍북,0.396524,0.830556,0.074169,...,0.552,3,1,2,1,5.0,7.0,3,1,2
139101,4,홍성읍,0.020,11,18,21,홍북,0.393365,0.837500,0.084399,...,0.579,3,1,2,1,5.0,7.0,3,1,2
139102,4,홍성읍,0.024,11,18,22,홍북,0.387046,0.806944,0.094629,...,0.593,3,1,2,1,5.0,7.0,3,1,2


In [78]:
test.to_csv(path_or_buf = path+'final_test.csv')

In [80]:
final_test = pd.read_csv(path+"final_test.csv")